In [4]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../title_maker_pro")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import re
import stanza
from collections import Counter
import itertools
import datasets
import pickle
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer
stanza.download('en')  

2020-05-03 18:55:13 INFO: Downloading default packages for language: en (English)...
2020-05-03 18:55:14 INFO: File exists: /home/tdimson/stanza_resources/en/default.zip.
2020-05-03 18:55:17 INFO: Finished downloading models and saved to /home/tdimson/stanza_resources.


In [ ]:
def print_words(words, f):
    for word in words:
        word_str = [word.word]
        if word.pos:
            word_str.append(f"/{word.pos}/")
        if word.topic:
            word_str.append(f"[{word.topic}]")
        print(" ".join(word_str), file=f)
        print(f"\t{word.definition}{' |n| ' if word.example is None else ''}", file=f)
        if word.example:
            print(f"\t\"{word.example}\"{' |e|' if word.from_example_expansion else ''}", file=f)

        print("", file=f)

In [6]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
blacklist = datasets.Blacklist.load("../build/blacklist.pickle")
model = AutoModelWithLMHead.from_pretrained("/mnt/evo/projects/title-maker-pro/models/en_dictionary_parsed_lr_00001/checkpoint-120000/").to("cuda:0")

2020-05-03 18:55:17 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-05-03 18:55:17 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-05-03 18:55:17 INFO: Use device: gpu
2020-05-03 18:55:17 INFO: Loading: tokenize
2020-05-03 18:55:19 INFO: Loading: pos
2020-05-03 18:55:19 INFO: Done loading processors!


In [21]:
%timeit datasets.ParsedDictionaryDefinitionDataset.evaluate_creativity(tokenizer, model, blacklist, 100, 50, max_length=512)

1.26 s ± 199 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def no_weird(w):
    return (
            w.word[-1] != "-"
            and "<|" not in w.definition
            and "<|" not in w.example
            and "<|" not in w.pos
            and len(word.word.split()) <= 3
        )
         

with torch.no_grad():
    words, stats = datasets.ParsedDictionaryDefinitionDataset.generate_words(
        tokenizer, model,
        num=10,
        max_iterations=1, 
        blacklist=blacklist, 
        do_example_expansion=False, 
        generation_args=dict(
            top_k=200,
            num_return_sequences=100,
            max_length=400,
            do_sample=True,
        ),
        expansion_generation_overrides=dict(
            top_k=50,
            num_return_sequences=30,
            do_sample=True,
        ),
        num_expansion_candidates=30,
        filter_proper_nouns=True,
        user_filter=no_weird,
    )

print(stats)
print()
#print_words(words, sys.stdout)

In [ ]:
import math
from transformers import activations
import transformers

def gelu_new(x):
    return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

activations.ACT2FN['gelu_new'] = gelu_new

model = AutoModelWithLMHead.from_pretrained("../build/forward-dictionary-model-v1").to("cpu")
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear, torch.nn.Embedding, transformers.modeling_utils.Conv1D}, dtype=torch.qint8
)

In [ ]:
a = go2()

In [ ]:
print(tokenizer.decode(a[0]))

In [ ]:
%timeit go2()

In [ ]:
from words import WordIndex, Word

In [ ]:
def clean_example(w, example):
    return re.sub(w, w, example, flags=re.IGNORECASE)
    
wi = WordIndex(
    [
        Word(
            word=w.word,
            definition=w.definition,
            pos=w.pos,
            topic=w.topic,
            example=clean_example(w.word, w.example),
        ) for w in words
    ]
)
wi.dump("../site/data/words.json")